In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w: i for i, w in enumerate(word_list)}


In [3]:
embedding_size =2
vocab_size = len(word_dict)
W = nn.Embedding(vocab_size, embedding_size)

In [4]:
inputs = torch.LongTensor([np.asarray([word_dict[n] for n in sen.split()]) for sen in sentences])

In [5]:
inputs

tensor([[ 3,  8,  7],
        [ 5, 10, 13],
        [ 2, 12, 14],
        [ 3, 15,  7],
        [ 0,  6,  4],
        [ 1,  9, 11]])

In [6]:
targets = torch.LongTensor([out for out in labels])
targets

tensor([1, 1, 1, 0, 0, 0])

In [7]:
w = W(inputs)
print(w.shape)

torch.Size([6, 3, 2])


In [8]:
w

tensor([[[-1.7486, -0.1385],
         [-0.1584, -0.1532],
         [ 0.3913,  0.2974]],

        [[ 0.6932,  2.3172],
         [ 0.2568,  0.9645],
         [-0.4114, -0.6245]],

        [[ 1.0114, -0.0720],
         [ 2.3788,  0.8022],
         [-1.5994, -0.7643]],

        [[-1.7486, -0.1385],
         [ 0.2028, -1.2965],
         [ 0.3913,  0.2974]],

        [[-0.5955,  1.4631],
         [ 0.4260, -1.0771],
         [ 0.4931,  0.6624]],

        [[ 1.1545, -1.1912],
         [ 0.5635,  0.5683],
         [ 0.5103, -0.5516]]], grad_fn=<EmbeddingBackward>)

In [9]:
w = w.unsqueeze(1)

In [10]:
w.shape

torch.Size([6, 1, 3, 2])

In [38]:
filter_sizes = [2, 2, 2]
num_filters = 3
filter_list = nn.ModuleList([nn.Conv2d(1, num_filters, (size, embedding_size)) for size in filter_sizes])


In [39]:
for i in filter_list:
    print(i)
    # 사용자가 지정한 필터를 돌려서 값이 나오는데 그걸 어떻게 하는거쥐?

Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))
Conv2d(1, 3, kernel_size=(2, 2), stride=(1, 1))


In [40]:
for i, conv in enumerate(filter_list):
    h = F.relu(conv(w))

In [41]:
h.shape

torch.Size([6, 3, 2, 1])

In [42]:
h

tensor([[[[0.7706],
          [0.0000]],

         [[0.0000],
          [0.0000]],

         [[1.2731],
          [0.4414]]],


        [[[0.0000],
          [0.0000]],

         [[0.0000],
          [0.0000]],

         [[0.0000],
          [0.3857]]],


        [[[0.0000],
          [0.0000]],

         [[0.0000],
          [0.4965]],

         [[0.0000],
          [0.0000]]],


        [[[1.4537],
          [0.1505]],

         [[0.0000],
          [0.0796]],

         [[1.5444],
          [0.3275]]],


        [[[0.4104],
          [0.0000]],

         [[0.0000],
          [0.1306]],

         [[0.8845],
          [0.1315]]],


        [[[0.0000],
          [0.0000]],

         [[0.4448],
          [0.0000]],

         [[0.0000],
          [0.2676]]]], grad_fn=<ReluBackward0>)

In [44]:
sequence_length = 3

mp = nn.MaxPool2d((sequence_length -filter_sizes[i] + 1, 1))
print(mp)
# 각 단어마다 거쳐서 나온 값이 2개이고 그 값에서 큰값을 도출 

MaxPool2d(kernel_size=(2, 1), stride=(2, 1), padding=0, dilation=1, ceil_mode=False)


In [45]:
a = mp(h)
print(a)
b = mp(h).permute(0, 3, 2, 1)
print(b)

tensor([[[[0.7706]],

         [[0.0000]],

         [[1.2731]]],


        [[[0.0000]],

         [[0.0000]],

         [[0.3857]]],


        [[[0.0000]],

         [[0.4965]],

         [[0.0000]]],


        [[[1.4537]],

         [[0.0796]],

         [[1.5444]]],


        [[[0.4104]],

         [[0.1306]],

         [[0.8845]]],


        [[[0.0000]],

         [[0.4448]],

         [[0.2676]]]], grad_fn=<MaxPool2DWithIndicesBackward>)
tensor([[[[0.7706, 0.0000, 1.2731]]],


        [[[0.0000, 0.0000, 0.3857]]],


        [[[0.0000, 0.4965, 0.0000]]],


        [[[1.4537, 0.0796, 1.5444]]],


        [[[0.4104, 0.1306, 0.8845]]],


        [[[0.0000, 0.4448, 0.2676]]]], grad_fn=<PermuteBackward>)


In [46]:
print(a.shape)
print(b.shape)

torch.Size([6, 3, 1, 1])
torch.Size([6, 1, 1, 3])


In [29]:
pooled_outputs = []
pooled_outputs.append(b)
print(pooled_outputs)

[tensor([[[[0.0000, 0.2461, 0.2706]]],


        [[[0.0000, 0.6341, 0.9572]]],


        [[[0.0000, 0.5048, 0.8070]]],


        [[[0.0000, 0.0000, 0.3058]]],


        [[[0.0000, 0.2506, 0.4929]]],


        [[[0.0000, 0.0000, 0.0000]]]], grad_fn=<PermuteBackward>)]


In [30]:
pooled_outputs[0][0][0][0]
# 6, 1, 1, 3 차원 

tensor([0.0000, 0.2461, 0.2706], grad_fn=<SelectBackward>)

In [31]:
filter_sizes = [2, 2, 2]
h_pool = torch.cat(pooled_outputs, len(filter_sizes))
h_pool

tensor([[[[0.0000, 0.2461, 0.2706]]],


        [[[0.0000, 0.6341, 0.9572]]],


        [[[0.0000, 0.5048, 0.8070]]],


        [[[0.0000, 0.0000, 0.3058]]],


        [[[0.0000, 0.2506, 0.4929]]],


        [[[0.0000, 0.0000, 0.0000]]]], grad_fn=<CatBackward>)

In [32]:
h_pool.shape

torch.Size([6, 1, 1, 3])

In [47]:
num_filters_total = num_filters * len(filter_sizes)
num_classes = 2
weight = nn.Linear(num_filters_total, num_classes, bias = False)
# num_classes가 output 

In [48]:
weight

Linear(in_features=9, out_features=2, bias=False)

In [49]:
h_pool_flat = torch.reshape(h_pool, [-1, num_filters_total])
h_pool_flat

tensor([[0.0000, 0.2461, 0.2706, 0.0000, 0.6341, 0.9572, 0.0000, 0.5048, 0.8070],
        [0.0000, 0.0000, 0.3058, 0.0000, 0.2506, 0.4929, 0.0000, 0.0000, 0.0000]],
       grad_fn=<ViewBackward>)

In [50]:
h_pool_flat.shape

torch.Size([2, 9])

In [53]:
bias = nn.Parameter(torch.ones([num_classes]))

In [55]:
bias

Parameter containing:
tensor([1., 1.], requires_grad=True)

In [57]:
weight(h_pool_flat)

tensor([[-0.7256,  0.2851],
        [-0.2527, -0.0055]], grad_fn=<MmBackward>)

In [54]:
weight(h_pool_flat)+bias
# 좀 이상한데 

tensor([[0.2744, 1.2851],
        [0.7473, 0.9945]], grad_fn=<AddBackward0>)